In [1]:
# https://htmx.org/examples/
# flask

In [2]:
%pip install flask


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask , Response , redirect
app = Flask("")#__name__)


# @app.route("/")
# def __HTMLRoot__():
#     global HTMLRoot
#     return ""

@app.route('/')
def hello():
    return redirect("//index.html", code=302)

get_resource = lambda:"<html><head/><body/></html>"
@app.route('/<path:path>')
def __get_resource( path ):  # pragma: no cover
    global get_resource
    return get_resource( path )

app.debug = False
launcher = lambda: [ app.run(host='127.0.0.1',port=18080) for x in range( 3 ) ]

from threading import Thread
Thread(target=launcher, args=() ).start()

 * Serving Flask app ''
 * Debug mode: off


 * Running on http://127.0.0.1:18080
Press CTRL+C to quit


127.0.0.1 - - [25/Oct/2023 00:10:37] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [25/Oct/2023 00:10:53] "GET /index.html HTTP/1.1" 200 -


### index.html


127.0.0.1 - - [25/Oct/2023 00:10:53] "GET /style.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 00:10:53] "GET /index.css HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 00:10:53] "GET /public/quote-background.svg HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 00:10:53] "GET /public/linkedin.svg HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 00:10:53] "GET /public/instagram.svg HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2023 00:10:53] "GET /public/twitter.svg HTTP/1.1" 200 -


### style.css
### index.css
### public/linkedin.svg
### public/quote-background.svg
### public/instagram.svg
### public/twitter.svg


In [ ]:
def get_resource( path ):  # pragma: no cover
    print("###" , path )
    mimetypes = {
        ".css": "text/css",
        ".html": "text/html",
        ".js": "application/javascript",
    }
    with open( path ) as f_io:
        return Response( f_io.read() , mimetype=mimetypes.get( path.split(".")[-1] , "text/html") )

In [2]:
# @app.root()
# def __HTMLRoot__(ur):
    # with open("index.html") as f_io:
        # return f_io.read()


AssertionError: The setup method 'route' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.

In [6]:
def HTMLRoot():
    with open("index.html") as f_io:
        return f_io.read()

In [7]:
HTMLRoot()

'<!DOCTYPE html>\n<html lang="en">\n  <head>\n    <title>Viwah_Quick</title>\n    <meta property="og:title" content="Viwah_Quick" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0" />\n    <meta charset="utf-8" />\n    <meta property="twitter:card" content="summary_large_image" />\n\n    <style data-tag="reset-style-sheet">\n      html {  line-height: 1.15;}body {  margin: 0;}* {  box-sizing: border-box;  border-width: 0;  border-style: solid;}p,li,ul,pre,div,h1,h2,h3,h4,h5,h6,figure,blockquote,figcaption {  margin: 0;  padding: 0;}button {  background-color: transparent;}button,input,optgroup,select,textarea {  font-family: inherit;  font-size: 100%;  line-height: 1.15;  margin: 0;}button,select {  text-transform: none;}button,[type="button"],[type="reset"],[type="submit"] {  -webkit-appearance: button;}button::-moz-focus-inner,[type="button"]::-moz-focus-inner,[type="reset"]::-moz-focus-inner,[type="submit"]::-moz-focus-inner {  border-style: none;  padding

127.0.0.1 - - [25/Oct/2023 00:04:19] "GET / HTTP/1.1" 200 -
[2023-10-25 00:04:19,766] ERROR in app: Exception on /style.css [GET]
Traceback (most recent call last):
  File "/home/codespace/.python/current/lib/python3.10/site-packages/flask/app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/codespace/.python/current/lib/python3.10/site-packages/flask/app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/codespace/.python/current/lib/python3.10/site-packages/flask/app.py", line 867, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/codespace/.python/current/lib/python3.10/site-packages/flask/app.py", line 852, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/tmp/ipykernel_32531/3500382234.py", line 21, in get_resource
    return Response( get_file(path) , mimetype=mimetypes.get( os.path.splitext(path)[1] , "text/html") )
NameErro